<a href="https://colab.research.google.com/github/darisoy/sudoku/blob/main/sudoku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import copy
from IPython.display import display, clear_output
import numpy as np
import pandas as pd
import random
import time
from tqdm import tqdm

In [2]:
def display_sudoku(board: np.ndarray):
    """Displays a 9x9 Sudoku board as a styled pandas DataFrame."""
    df = pd.DataFrame(board).astype(str).replace('0', '.')

    styler = df.style.set_properties(**{
        'font-size': '16pt',
        'text-align': 'center',
        'width': '25px',
        'height': '25px',
    }).hide(axis='index').hide(axis='columns')

    styler.set_table_styles([
        {'selector': 'tr:nth-child(3n)', 'props': 'border-bottom: 5px solid black;'},
        {'selector': 'td:nth-child(3n)', 'props': 'border-right: 5px solid black;'},
        {'selector': 'tr:first-child', 'props': 'border-top: 5px solid black;'},
        {'selector': 'td:first-child', 'props': 'border-left: 5px solid black;'},
        {'selector': 'td', 'props': 'border: 0.5px solid #d3d3d3;'} # Light gray for inner lines
    ], overwrite=False)

    display(styler)

In [3]:
def is_valid(puzzle: np.ndarray, num: int, row: int, col: int) -> bool:
    return num not in puzzle[row, :] and \
       num not in puzzle[:, col] and \
       num not in puzzle[(row // 3) * 3:(row // 3) * 3 + 3, (col // 3) * 3:(col // 3) * 3 + 3]

def solve_sudoku_recursive(puzzle: np.ndarray) -> bool:
    """
    Recursive helper function to solve the sudoku puzzle in-place.
    Returns True if a solution is found, False otherwise.
    """
    # Find the next empty cell (value 0)
    for row in range(9):
        for col in range(9):
            if puzzle[row, col] != 0:
                continue
            # Try placing numbers 1-9 in a random order
            for num in np.random.permutation(np.arange(1, 10)):
                if is_valid(puzzle, num, row, col):
                    # If valid, place the number
                    puzzle[row, col] = num

                    # Recurse to solve the rest of the puzzle
                    if solve_sudoku_recursive(puzzle):
                        return True # Success!

                    # If recursion failed, backtrack and try the next number
                    puzzle[row, col] = 0

            # If no number from 1-9 worked, trigger backtracking
            return False

    # If the board has no empty cells, it's solved
    return True

def generate_solution() -> np.ndarray:
    """
    Generates a new, fully solved Sudoku board using an optimized
    recursive backtracking algorithm.
    """
    # 1. Start with an empty 9x9 board
    puzzle = np.zeros((9, 9), dtype=int)

    # 2. Solve the puzzle in-place
    solve_sudoku_recursive(puzzle)

    return puzzle

solved_board = generate_solution()
display_sudoku(solved_board)

6,2,9,4,5,7,8,3,1
3,7,1,8,9,6,2,5,4
5,8,4,1,2,3,7,6,9
7,6,2,3,4,1,5,9,8
1,4,5,9,6,8,3,7,2
8,9,3,5,7,2,1,4,6
4,1,7,2,3,9,6,8,5
2,5,6,7,8,4,9,1,3
9,3,8,6,1,5,4,2,7


In [4]:
def _solve_and_count(puzzle: np.ndarray, count: list) -> None:
    """
    Recursive helper to find and count all possible solutions.
    It stops early if more than one solution is found.
    """
    for row in range(9):
        for col in range(9):
            if puzzle[row, col] == 0:
                for num in range(1, 10):
                    if is_valid(puzzle, num, row, col):
                        puzzle[row, col] = num
                        _solve_and_count(puzzle, count)
                        puzzle[row, col] = 0 # Backtrack
                return

    # full solution is found
    count[0] += 1

def count_solutions(puzzle: np.ndarray) -> int:
    """Counts the number of solutions for a given Sudoku board."""
    # Use a list for the counter so it's mutable across recursive calls
    solution_count = [0]
    _solve_and_count(puzzle.copy(), solution_count)
    return solution_count[0]


# --- Main Puzzle Creation Function ---

def make_puzzle(solution: np.ndarray, n: int) -> np.ndarray:
    """
    Generates a puzzle from a solved board by removing n elements.

    Args:
        solution: A 9x9 NumPy array representing a fully solved Sudoku board.
        n: The number of elements to remove.

    Returns:
        A 9x9 NumPy array representing the puzzle with a unique solution.
    """
    # Start with a copy of the full solution
    puzzle = solution.copy()

    # Get a shuffled list of all cell coordinates (0,0) to (8,8)
    coords = np.argwhere(puzzle > 0)
    np.random.shuffle(coords)

    removed_count = 0

    # Try to remove numbers one by one from the shuffled coordinates
    for row, col in coords:
        # Stop when we have removed the desired number of cells
        if removed_count >= n:
            break

        # Store the original value in case we need to put it back
        original_value = puzzle[row, col]
        puzzle[row, col] = 0

        # Check if the puzzle still has exactly one solution
        num_solutions = count_solutions(puzzle)

        if num_solutions == 1:
            # The removal was successful, increment our counter
            removed_count += 1
        else:
            # The removal made the puzzle invalid (0 or >1 solutions), so undo it
            puzzle[row, col] = original_value

    return puzzle

In [5]:
class Sudoku:
    def __init__(self, n: int):
        self.solution = generate_solution()
        self.puzzle = make_puzzle(self.solution, n)

    def __str__(self) -> str:
        return str(self.puzzle)

In [6]:
def solve1(puzzle: np.ndarray) -> None:
    s = copy.deepcopy(puzzle)
    to_solve = np.argwhere(puzzle == 0)

    orig_possible = []
    for ts in to_solve:
        x, y = ts[0], ts[1]
        new_possible = []
        for num in range(1, 10):
            if is_valid(puzzle, num, x, y):
                new_possible.append(num)
        orig_possible.append(new_possible)

    possible = copy.deepcopy(orig_possible)
    i = 0
    while i >= 0 and i < len(to_solve):
        x, y = to_solve[i][0], to_solve[i][1]
        num = 0
        for p in possible[i]:
            if is_valid(s, p, x, y):
                num = p
                break
        if num == 0: # no solution, need to backtrack
            s[x, y] = 0
            possible[i] = copy.deepcopy(orig_possible[i])
            i -= 1
        else:
            s[x, y] = num
            possible[i].remove(num)
            i += 1
    return s

s = Sudoku(30)
solved = solve1(s.puzzle)

display_sudoku(s.puzzle)

print(np.array_equal(s.solution, solved))

.,1,.,.,5,.,3,.,2
3,5,.,.,4,1,9,7,.
9,.,6,2,3,8,5,1,4
5,2,.,1,6,4,7,8,9
6,.,1,.,.,7,.,.,5
8,9,.,5,.,3,1,4,.
.,8,.,.,9,.,.,5,.
2,3,.,8,1,.,4,9,7
.,.,9,4,.,5,8,.,3


True


In [7]:
def solve2(puzzle: np.ndarray) -> None:
    s = copy.deepcopy(puzzle)
    to_solve = np.argwhere(s == 0)
    orig_possible = []
    for ts in to_solve:
        x, y = ts[0], ts[1]
        new_possible = []
        for num in range(1, 10):
            if is_valid(puzzle, num, x, y):
                new_possible.append(num)
        if len(new_possible) == 1:
            s[x, y] = new_possible[0]
        else:
            orig_possible.append(new_possible)

    to_solve = np.argwhere(s == 0)
    possible = copy.deepcopy(orig_possible)
    i = 0
    while i >= 0 and i < len(to_solve):
        x, y = to_solve[i][0], to_solve[i][1]
        num = 0
        for p in possible[i]:
            if is_valid(s, p, x, y):
                num = p
                break
        if num == 0: # no solution, need to backtrack
            s[x, y] = 0
            possible[i] = copy.deepcopy(orig_possible[i])
            i -= 1
        else:
            s[x, y] = num
            possible[i].remove(num)
            i += 1
    return s

question = np.array([
    [0,1,9, 2,0,0, 0,0,6],
    [0,0,0, 0,0,0, 5,0,1],
    [0,0,0, 0,1,0, 9,0,0],

    [0,4,5, 0,8,0, 0,0,0],
    [6,0,0, 0,9,0, 0,0,4],
    [0,0,0, 0,2,0, 6,8,0],

    [0,0,1, 0,3,0, 0,0,0],
    [2,0,7, 0,0,0, 0,0,0],
    [5,0,0, 0,0,9, 7,3,0],
    ])
result = solve2(question)
print()
display_sudoku(result)

3,1,9,2,7,5,8,4,6
8,7,6,9,4,3,5,2,1
4,5,2,6,1,8,9,7,3
1,4,5,3,8,6,2,9,7
6,2,8,5,9,7,3,1,4
7,9,3,4,2,1,6,8,5
9,6,1,7,3,2,4,5,8
2,3,7,8,5,4,1,6,9
5,8,4,1,6,9,7,3,2


In [13]:
def solve3(puzzle: np.ndarray) -> None:
    s = puzzle.copy()
    to_solve = np.argwhere(s == 0)
    orig_possible = []
    for ts in to_solve:
        x, y = ts[0], ts[1]
        new_possible = []
        for num in range(1, 10):
            if is_valid(puzzle, num, x, y):
                new_possible.append(num)
        if len(new_possible) == 1:
            s[x, y] = new_possible[0]
        else:
            orig_possible.append(new_possible)

    to_solve = np.argwhere(s == 0)
    possible = copy.deepcopy(orig_possible)
    i = 0
    while i >= 0 and i < len(to_solve):
        x, y = to_solve[i][0], to_solve[i][1]
        num = 0
        for p in possible[i]:
            if is_valid(s, p, x, y):
                num = p
                break
        if num == 0: # no solution, need to backtrack
            s[x, y] = 0
            possible[i] = orig_possible[i].copy()
            i -= 1
        else:
            s[x, y] = num
            possible[i].remove(num)
            i += 1
    return s

question = np.array([
    [0,1,9, 2,0,0, 0,0,6],
    [0,0,0, 0,0,0, 5,0,1],
    [0,0,0, 0,1,0, 9,0,0],

    [0,4,5, 0,8,0, 0,0,0],
    [6,0,0, 0,9,0, 0,0,4],
    [0,0,0, 0,2,0, 6,8,0],

    [0,0,1, 0,3,0, 0,0,0],
    [2,0,7, 0,0,0, 0,0,0],
    [5,0,0, 0,0,9, 7,3,0],
    ])
result = solve3(question)
print()
display_sudoku(result)

3,1,9,2,7,5,8,4,6
8,7,6,9,4,3,5,2,1
4,5,2,6,1,8,9,7,3
1,4,5,3,8,6,2,9,7
6,2,8,5,9,7,3,1,4
7,9,3,4,2,1,6,8,5
9,6,1,7,3,2,4,5,8
2,3,7,8,5,4,1,6,9
5,8,4,1,6,9,7,3,2


In [30]:
def solve4(puzzle: np.ndarray) -> None:
    s = puzzle.copy()

    to_solve = np.argwhere(s == 0)
    orig_possible = []
    for ts in to_solve:
        x, y = ts[0], ts[1]
        new_possible = []
        for num in range(1, 10):
            if is_valid(puzzle, num, x, y):
                new_possible.append(num)
        if len(new_possible) == 1:
            s[x, y] = new_possible[0]
        else:
            orig_possible.append(new_possible)

    to_solve = np.argwhere(s == 0)
    for i, ts in enumerate(to_solve):
        x, y = ts[0], ts[1]
        p = []
        for op in orig_possible[i]:
            if is_valid(s, op, x, y):
                p.append(op)
        orig_possible[i] = p


    possible = copy.deepcopy(orig_possible)
    i = 0
    while i >= 0 and i < len(to_solve):
        x, y = to_solve[i][0], to_solve[i][1]
        num = 0
        for p in possible[i]:
            if is_valid(s, p, x, y):
                num = p
                break
        if num == 0: # no solution, need to backtrack
            s[x, y] = 0
            possible[i] = orig_possible[i].copy()
            i -= 1
        else:
            s[x, y] = num
            possible[i].remove(num)
            i += 1
    return s

question = np.array([
    [0,1,9, 2,0,0, 0,0,6],
    [0,0,0, 0,0,0, 5,0,1],
    [0,0,0, 0,1,0, 9,0,0],

    [0,4,5, 0,8,0, 0,0,0],
    [6,0,0, 0,9,0, 0,0,4],
    [0,0,0, 0,2,0, 6,8,0],

    [0,0,1, 0,3,0, 0,0,0],
    [2,0,7, 0,0,0, 0,0,0],
    [5,0,0, 0,0,9, 7,3,0],
    ])
result = solve4(question)
print()
display_sudoku(result)

3,1,9,2,7,5,8,4,6
8,7,6,9,4,3,5,2,1
4,5,2,6,1,8,9,7,3
1,4,5,3,8,6,2,9,7
6,2,8,5,9,7,3,1,4
7,9,3,4,2,1,6,8,5
9,6,1,7,3,2,4,5,8
2,3,7,8,5,4,1,6,9
5,8,4,1,6,9,7,3,2


In [110]:
def solve5(puzzle: np.ndarray) -> None:
    s = puzzle.copy()
    possibilities = np.ones((9, 9, 9), dtype=bool)
    prev_zeros = 81
    while prev_zeros > len(np.argwhere(s == 0)):
        # print(len(np.argwhere(s == 0)))
        prev_zeros = len(np.argwhere(s == 0))
        if len(np.argwhere(s == 0)) == 0:
            return s
        for (x, y, n), p in np.ndenumerate(possibilities):
            if s[x, y] == 0:
                possibilities[x, y, n] = is_valid(s, n+1, x, y)
        to_solve = np.argwhere(s == 0)
        for ts in to_solve:
            x, y = ts[0], ts[1]
            if possibilities[x, y].sum() == 1:
                s[x, y] = np.where(possibilities[x, y])[0][0] + 1

    to_solve = np.argwhere(s == 0)
    new_possibilities = possibilities.copy()
    i = 0
    while i >= 0 and i < len(to_solve):
        x, y = to_solve[i][0], to_solve[i][1]
        num = 0
        for n in np.argwhere(new_possibilities[x, y]):
            n = n[0]
            if is_valid(s, n+1, x, y):
                num = n+1
                break
        if num == 0: # no solution, need to backtrack
            s[x, y] = 0
            new_possibilities[x, y] = possibilities[x, y].copy()
            i -= 1
        else:
            s[x, y] = num
            new_possibilities[x, y, num-1] = False
            i += 1
    return s

sudoku = Sudoku(50)
result = solve5(sudoku.puzzle)
print()
display_sudoku(result)
print(np.array_equal(sudoku.solution, result))

2,8,9,6,7,5,1,4,3
7,6,1,3,4,8,9,5,2
4,5,3,9,2,1,6,8,7
5,7,4,8,3,9,2,6,1
9,1,8,5,6,2,7,3,4
6,3,2,7,1,4,8,9,5
8,4,6,1,5,7,3,2,9
1,9,5,2,8,3,4,7,6
3,2,7,4,9,6,5,1,8


True


In [120]:
def solve6(puzzle: np.ndarray) -> np.ndarray | None:
    s = puzzle.copy()
    possibilities = np.ones((9, 9, 9), dtype=bool)
    for x, y in np.argwhere(s > 0):
        _update_possibilities(possibilities, x, y, s[x, y])

    # Fill in all cells that have only one possible value.
    while True:
        single_cells = np.argwhere((possibilities.sum(axis=2) == 1) & (s == 0))
        if single_cells.size == 0:
            break
        for x, y in single_cells:
            n = np.where(possibilities[x, y])[0][0] + 1
            s[x, y] = n
            _update_possibilities(possibilities, x, y, n)

    stack = [(s, possibilities)]
    while stack:
        s, possibilities = stack.pop()

        unsolved = np.argwhere(s == 0)
        if unsolved.size == 0:
            return s

        # Find the empty cell with the fewest possibilities.
        x, y = unsolved[np.argmin(possibilities.sum(axis=2)[s == 0])]

        # Iterate through possible numbers for the chosen cell.
        for num_idx in reversed(np.where(possibilities[x, y])[0]):
            num = num_idx + 1

            # Create a new state for this guess
            next_s = s.copy()
            next_possibilities = possibilities.copy()
            next_s[x, y] = num
            _update_possibilities(next_possibilities, x, y, num)

            # Push the new state onto the stack to be explored.
            stack.append((next_s, next_possibilities))

    return None # no solution found.

def _update_possibilities(possibilities: np.ndarray, x: int, y: int, n: int):
    possibilities[x, :, n-1] = False
    possibilities[:, y, n-1] = False
    box_x, box_y = x // 3 * 3, y // 3 * 3
    possibilities[box_x:box_x+3, box_y:box_y+3, n-1] = False
    possibilities[x, y, :] = False
    possibilities[x, y, n-1] = True

sudoku = Sudoku(50)
result = solve6(sudoku.puzzle)
print()
display_sudoku(result)
print(np.array_equal(sudoku.solution, result))

1,3,6,8,4,7,2,9,5
4,9,5,2,3,1,8,6,7
8,2,7,9,5,6,3,4,1
3,7,8,4,1,5,6,2,9
2,4,9,7,6,8,1,5,3
6,5,1,3,2,9,7,8,4
9,8,3,6,7,4,5,1,2
7,1,4,5,8,2,9,3,6
5,6,2,1,9,3,4,7,8


True


In [18]:
n = 500
sudokus = [Sudoku(random.randint(15, 45)) for _ in range(n)]

In [141]:
def performance_test(f):
    duration = 0
    for i in tqdm(range(n), desc="Processing items"):
        start_time = time.perf_counter()
        solved = f(sudokus[i].puzzle)
        duration += (time.perf_counter() - start_time) * 1000 * 1000
        if not np.array_equal(sudokus[i].solution, solved):
            print("Puzzle not solved!")
    print(f'\nSolved {n} puzzles in {duration / n:.2f} us on average.')

performance_test(solve1)
performance_test(solve2)
performance_test(solve3)
performance_test(solve4)
performance_test(solve5)
performance_test(solve6)

Processing items: 100%|██████████| 500/500 [00:02<00:00, 216.45it/s]



Solved 500 puzzles in 4569.67 us on average.


Processing items: 100%|██████████| 500/500 [00:01<00:00, 281.53it/s]



Solved 500 puzzles in 3508.13 us on average.


Processing items: 100%|██████████| 500/500 [00:01<00:00, 283.54it/s]


...


Solved 500 puzzles in 3946.61 us on average.


Processing items: 100%|██████████| 500/500 [00:05<00:00, 94.55it/s] 



Solved 500 puzzles in 10363.66 us on average.


Processing items: 100%|██████████| 500/500 [00:00<00:00, 1206.13it/s]


Solved 500 puzzles in 812.90 us on average.
